In [ ]:
# Import necessary libraries
import urllib.request
import importlib.util
!pip install mglearn
import mglearn
url = "https://raw.githubusercontent.com/brighamfrandsen/econ484/refs/heads/master/utilities/preamble.py"
urllib.request.urlretrieve(url, "preamble.py")
spec = importlib.util.spec_from_file_location("preamble", "preamble.py")
preamble = importlib.util.module_from_spec(spec)
spec.loader.exec_module(preamble)
from preamble import *

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brighamfrandsen/econ484/blob/master/examples/movies.ipynb)

# Example: Movie-going and Weather

This notebook will illustrate the entire supervised machine learning process in the context of predicting movie attendance based on the weather on opening weekend.

## 1. Figure out your question

How many people would be expected to attend a movie on a weekend with temperatures in the X1s, precipitation of X2, humidity of X3, . . . ?

## 2. Obtain a labeled dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
moviedata=pd.read_csv('https://github.com/brighamfrandsen/econ484/blob/master/data/opening_wkend.csv?raw=true')

print(moviedata.head())
print("Shape: {}".format(str(moviedata.shape)))

   tickets_wk1d_r  tickets_ratedgpg_wk1d_r  tickets_adult_wk1d_r  \
0       -1.718881                -0.710163             -1.603236   
1       -1.311943                      NaN             -0.815862   
2        0.958356                      NaN              0.979913   
3       -1.305518                      NaN             -1.066690   
4        2.094907                -0.010072              1.653934   

   tickets_p33_highbudget_wk1d_r  tickets_p33_lowbudget_wk1d_r  \
0                            NaN                     -0.704108   
1                            NaN                           NaN   
2                       0.546590                           NaN   
3                      -0.974721                           NaN   
4                       1.725840                           NaN   

   tickets_p33_hr1000_wk1d_r  tickets_p33_lr1000_wk1d_r  tickets_pt_wk1d_r  \
0                        NaN                        NaN          -2.218872   
1                        NaN          

Let's define our "label" (y) vector and our "feature" matrix (X):

In [ ]:
y = moviedata.filter(items=['tickets_wk1d_r'])
X = moviedata.filter(like='res_own',axis=1)
print('our y vector is:\n',y.head)
print('our X matrix is:\n',X.head)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

our y vector is:
 <bound method NDFrame.head of       tickets_wk1d_r
0          -1.718881
1          -1.311943
2           0.958356
3          -1.305518
4           2.094907
...              ...
1666        2.575074
1667        6.183819
1668       -2.352185
1669       -2.064675
1670       -1.981390

[1671 rows x 1 columns]>
our X matrix is:
 <bound method NDFrame.head of       res_own_snow  res_own_rain  res_own_mat5_10  res_own_mat5_15  \
0        -0.115566     -0.084558        -0.024600        -0.027172   
1         0.041614     -0.100350         0.002211         0.001529   
2         0.065586      0.139913         0.001574         0.001651   
3         0.000226     -0.102481         0.001544         0.001651   
4        -0.000154     -0.041737         0.001577         0.001673   
...            ...           ...              ...              ...   
1666     -0.031120     -0.062038         0.004453         0.002085   
1667     -0.033789     -0.114728         0.003818         0.002019

## 3. Divide into training and test sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=.25)
y_train.shape

(1253, 1)

## 4. Pick an appropriate method

In [ ]:
from sklearn.linear_model import Lasso

## 5. Choose regularization parameters via cross-validation on the training set

By hand if you really want:

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
lasso = Lasso(alpha=0.1, max_iter=100)
scores = cross_val_score(lasso,X_train,y_train,cv=5)
print("Cross-validation scores: {}".format(scores))
print("Average cross-validation score: {:.4f}".format(scores.mean()))

Cross-validation scores: [ 0.00974058 -0.0135029   0.01768621  0.01618713  0.02159627]
Average cross-validation score: 0.0103


Or use GridSearchCV and do it automatically:

In [ ]:
from sklearn.model_selection import GridSearchCV
# define grid for alpha
alpha_grid = {'alpha': [.0001, .005,.01, .015,.02, .025, .03, .031, .032, .034, .035, .036, .04 ,.05, .06 ],'max_iter': [1000]}
grid_search = GridSearchCV(Lasso(),alpha_grid,cv=5,return_train_score=True)
best_model=grid_search.fit(X_train,y_train)
print("Best alpha: ",best_model.best_estimator_.get_params()['alpha'])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.718e+02, tolerance: 1.358e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.859e+02, tolerance: 1.374e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.794e+02, tolerance: 1.365e

Best alpha:  0.032


Or, even easier, just use LassoCV:

In [ ]:
from sklearn.linear_model import LassoCV
lassocv = LassoCV(cv=5,max_iter=1000).fit(X_train, y_train)
print(lassocv.alpha_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.21119559708586166, tolerance: 0.13580844599209113
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.17314731759279312, tolerance: 0.13580844599209113
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Object

0.03162543390583107


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 114.2567875136217, tolerance: 0.14134953469485165
  model = cd_fast.enet_coordinate_descent_gram(


## 6. Fit model on whole training set using the cross-validated parameters

In [ ]:
lassowcvalpha=Lasso(alpha = lassocv.alpha_,max_iter=100000).fit(X_train,y_train)

In [ ]:
lassowcvalpha.coef_

array([-0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        ,  0.        , -0.        ,  0.        ,  0.        ,
       -0.        ,  0.        ,  0.        ,  0.        , -0.        ,
       -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.02361798, -0.        , -0.        ,
        0.        , -0.00181429,  0.        ,  0.        , -0.        ,
       -0.        , -0.        ,  0.        ,  0.        , -0.        ,
       -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
       -0.        , -0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        ,  0.        ,  0.        , -0.05170246,
        0.00968026,  0.00128332,  0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.00684583,
       -0.        , -0.        ,  0.        , -0.        , -0.  

## 7. Evaluate model by applying it to test set

In [ ]:
print('Lasso score on test set: {:.4f}'.format(lassowcvalpha.score(X_test,y_test)))

Lasso score on test set: 0.0543


## 8. Repeat 4-7 for several methods

## 9. Apply to new observations for which we have no labels

In [ ]:
Xnew=pd.read_csv('https://github.com/brighamfrandsen/econ484/blob/master/data/newobs.csv?raw=true')

# standardize new Xs in same way
Xnew = scaler.transform(Xnew)
yhatnew=lassowcvalpha.predict(Xnew)
print("predicted residualized ticket sales for new observation: ",yhatnew)

predicted residualized ticket sales for new observation:  [-0.09059448]


In [ ]:
lassowcvalpha.coef_